# The Web as a Knowledge-base for Answering Complex Questions

Alon Talmor, Jonathan Berant (NAACL 2018)

### Past work

* Neural RC Methods:
    * Near human-level performance
    * They mostly excel at matching questions to local contexts, but struggle with questions that require reasoning
    * RC assumes documents with the information relevant for the answer are available
* QA
    * *Compositionality*: Questions are translated to compositional programs that encode a sequence of actions for finding the answer in a knowledge-base (KB)
    * This reliance on a manually-curated KB has limited the coverage and applicability of semantic parsers

### Contributions:

* framework for answering complex questions through question decomposition
* sequence-to-sequence model for question decomposition
* [dataset](https://www.tau-nlp.org/compwebq) of 34,689 examples of complex and broad questions, along with answers, web snippets, and SPARQL queries

### Assumption

**answering simple questions can be achieved by combining a search engine with a RC model**

### Problem formulation

The authors assume a pretrained, blackbox model ${\rm S{\scriptsize IMP}QA}$ which works by 

1. submitting the question to a search engine that retrieves web snippets
2. using an RC model to extract answer from the snippets

The proposed model decomposes a question *q* into a computation tree *t*. A computation tree has (paraphrased) strings as leaves and functions at inner nodes. An answer is computed by recursively applying the function at the root to its child subtrees (function at leaf is the identity). Available functions:

1. ${\rm S{\scriptsize IMP}QA}(.)$: Takes a string argument and returns the answer
2. ${\rm C{\scriptsize OMP}}(.,.)$: Takes a string with a variable VAR and a set of of answers.
$${\rm C{\scriptsize OMP}}(q, \mathcal{A}) = \cup_{a\in\mathcal{A}} {\rm S{\scriptsize IMP}QA}(q/a)$$ where q/a denotes the string produced when replacing VAR in q with a
3. ${\rm C{\scriptsize ONJ}}(.,.)$: Takes two sets and returns their intersection. Other set operations can be defined analogously.
4. ${\rm A{\scriptsize DD}}(·,·)$: Takes two singleton sets of numbers and returns a set with their addition. Similar mathematical operations can be defined analogously.

### Dataset collection

Based on and extends [${\rm W{\scriptsize EB}Q{\scriptsize UESTIONS}SP}$](http://aka.ms/WebQSP) (Yih et al., 2016) which contains 4,737 questions paired with SPARQL queries for Freebase

1. sample question-query pairs
2. automatically create more complex SPARQL queries (conjunctions, superlatives, comparatives, and compositions)
3. generate automatically questions (MG) that are understandable to Amazon Mechanical Turk (AMT) workers
4. have AMT workers paraphrase MG questions into natural language (workers incentivized to create questions with high edit distance compared to the MG question)

Each of our examples contains a question, an answer, a SPARQL query (that our models ignore), and all web snippets harvested by our model when attempting to answer the question

### Proposed model (simplified)

1. Use Seq2SQL type model to convert query into a tree as mentioned before
    * Noisy supervision to train the model is generated by heuristically working from the original SPARQL query to the natural language question
2. Apply the generated tree ops to the query to get a set of "simple" questions
3. Pass the questions to a ${\rm S{\scriptsize IMP}QA}(.)$ function. 
    > This model sends the question to Google’s search engine and extracts a distribution over answers from the top-100 web snippets using manually-engineered features. We re-train the model on our data with one new feature: for every question *q* and candidate answer mention in a snippet, we run [RASOR](https://arxiv.org/abs/1611.01436), an RC model by lee et al. (2016), and add the output logit score as a feature. We found that combining the web-facing model of Talmor et al. (2017) and RASOR, resulted in improved performance.

**Note:**
* Authors perform a detailed analysis of the model performance on various types of questions. However only performance changes caused tweaks to their proposed model are mentioned. As a baseline, they use a model trained for direct question answering (basically SimpQA on the whole query)
* Authors also show good performance on the original ComplexQuestions dataset. Compared with a model trained directly on the source database (FreeBase), the authors claim that the proposed architecture performs comparably.

**Observations:**
* The Seq2SQL model is trained in a constrained manner and can only perform one operation (out of Comp, Conj and SimpQA) for query decomposition. This is an artifact from data generation because the authors know that the questions were made from atmost 2 decomposable sub-questions